<a href="https://colab.research.google.com/github/SainiManisha/dbms-tutorial/blob/master/TODO_List_Basic_Flask_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask flask-ngrok mysql-connector

In [1]:
import mysql.connector
import pandas as pd
from flask import Flask, request
from flask_ngrok import run_with_ngrok

In [2]:
conn = mysql.connector.connect(
    host="34.93.147.110",
    user="root",
    password="123456789",
    port=3306,
    database='webapp'
)
cur = conn.cursor()

In [3]:
query1 = """DROP TABLE IF EXISTS todo;"""
query2 = """
    CREATE TABLE todo (
        id INT(10) AUTO_INCREMENT, 
        name TEXT NOT NULL,
        email VARCHAR(50),
        task TEXT,
        tag TEXT,
        status VARCHAR(10) DEFAULT 'notdone',
        CHECK(status IN ('done', 'notdone')),
        PRIMARY KEY(id)
    )
"""
cur.execute(query1)
conn.commit()

cur.execute(query2)
conn.commit()

In [4]:
def insert(name, email, task, tag, done=False):
    status = 'done' if done else 'notdone'
    query = """
        INSERT INTO todo (name, email, task, tag, status) 
        VALUES (%s, %s, %s, %s, %s)"""
    cur.execute(query, (name, email, task, tag, status))
    conn.commit()

In [5]:
insert("Manisha", "hi@bye.com", "Buy groceries", "home", done=False)
insert("Manisha", "hi@bye.com", "New shoes", "home", done=True)

In [16]:
app = Flask(__name__)
run_with_ngrok(app)

In [17]:
@app.route('/')
def show():
    query = """
        SELECT * FROM todo;"""
    df = pd.read_sql(query, conn)
    return """<h1> My ToDo List</h1>""" + 
    df.to_html() + 
    """<a href="/form">Add to List</a>"""

In [18]:
@app.route('/form')
def form():
    html = """
    <form action='/submit' method='POST'>
        <input type='text' placeholder='name' name='name'> <br>
        <input type='email' placeholder='email' name='email'> <br>
        <input type='text' placeholder='task' name='task'> <br>
        <input type='text' placeholder='tag' name='tag'> <br>
        <input type="radio" name="status" value="done" id="done">
        <label for="done">Done</label><br>
        <input type="radio" name="status" value="not done" id="notdone">
        <label for="notdone">Not done</label><br>
        <input type='submit'>
    </form>
    """
    return html

In [19]:
@app.route('/submit', methods=['POST'])
def submit():
    name = request.form['name']
    email = request.form['email']
    task = request.form['task']
    tag = request.form['tag']
    done = True if request.form['status'] == 'done' else False

    insert(name, email, task, tag, done)
    return """Thank you for your todo list addition.<br>
            <a href='/'>Show my TODO list</a>"""

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://623e3ee0c71e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Oct/2020 09:16:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 09:16:54] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Oct/2020 09:16:55] "GET /form HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 09:17:06] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 09:17:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 09:20:44] "GET /form HTTP/1.1" 200 -
